In [ ]:
import sys
sys.path.append("../")
import numpy as np
import h5py
import torch as pt
from tqdm import tqdm
import matplotlib.pyplot as plt
from LSTM_construction import CTLSTM
import os

def prepare_bucket_tensors(bucket_data, N_types=3):
    """
    Convertit les données d'un bucket en tenseurs PyTorch
    """
    EventsData = bucket_data['EventsData']
    SeqLengthData = bucket_data['SeqLengthData']
    timesData = bucket_data['timesData']
    timeMaxData = bucket_data['timeMaxData']
    
    N_train = EventsData.shape[0]
    N_seq_Max = EventsData.shape[1]
    
    # Créer les événements one-hot
    Events_one_hot = np.zeros((N_train, N_seq_Max, N_types))
    for seq in range(N_train):
        for step in range(SeqLengthData[seq]):
            ev = EventsData[seq, step]
            if ev >= 0:  # Événement valide
                Events_one_hot[seq, step, ev] = 1.0
    
    # Convertir en tenseurs PyTorch
    timeScale = 1.0
    EvTens = pt.tensor(Events_one_hot).double()
    EvIndTens = pt.tensor(EventsData).long()
    timeTensor = pt.tensor(timesData/timeScale).double()
    tMaxTensor = pt.tensor(timeMaxData/timeScale).double()
    mask = EvIndTens.ge(-1+0.001)
    
    return EvTens, EvIndTens, timeTensor, tMaxTensor, mask, SeqLengthData

def evaluate_on_validation(net, EvTens_dev, EvIndTens_dev, timeTensor_dev, tMaxTensor_dev, mask_dev, batch_size=10):
    """
    Évalue le modèle sur les données de validation
    """
    net.eval()
    total_dev_loss = 0.0
    N_dev = len(EvTens_dev)
    N_seq_Max = EvTens_dev.shape[1]
    
    with pt.no_grad():
        for batchInd in range(0, N_dev, batch_size):
            batch_end = min(batchInd + batch_size, N_dev)
            BatchEventsHot_dev = EvTens_dev[batchInd:batch_end]
            BatchEventsInd_dev = EvIndTens_dev[batchInd:batch_end]
            BatchTimes_dev = timeTensor_dev[batchInd:batch_end]
            BatchTMax_dev = tMaxTensor_dev[batchInd:batch_end]
            BatchMask_dev = mask_dev[batchInd:batch_end]
            
            # Forward pass
            lambOuts_dev, CLows_dev, Cbars_dev, deltas_dev, OutGates_dev = net.forward(
                BatchEventsHot_dev, BatchMask_dev, BatchTimes_dev
            )
            
            # Calculate MC loss
            LMC_dev, _, _, _ = net.MC_Loss(
                BatchTimes_dev, BatchTMax_dev, CLows_dev, Cbars_dev, 
                deltas_dev, OutGates_dev, Nsamples=N_seq_Max
            )
            
            # Calculate likelihood loss
            LogLikeLoss_dev, _ = net.logLoss(BatchEventsInd_dev, BatchMask_dev, lambOuts_dev)
            
            loss_dev = LogLikeLoss_dev + LMC_dev
            total_dev_loss += loss_dev.item()*(BatchEventsInd_dev.shape[0])
    
    return total_dev_loss

def train_bucket(net, optimizer, EvTens, EvIndTens, timeTensor, tMaxTensor, mask, 
                SeqLengthData, EvTens_val, EvIndTens_val, timeTensor_val, tMaxTensor_val, mask_val, SeqLengthData_val,
                bucket_idx, num_epochs=5, batch_size=10):
    """
    Entraîne le modèle sur un bucket pour un nombre fixe d'époques
    MODIFICATION: Calcule la validation loss après chaque époque
    """
    N_train = len(EvTens)
    N_seq_Max = EvTens.shape[1]
    bucket_train_losses = []
    bucket_val_losses = []
    sequence_losses_per_epoch = {}
    
    for epoch in range(num_epochs):
        net.train()
        epoch_loss = 0.0
        epoch_total_losses = []
        epoch_losses_dict = {}
        
        # Permutation pour cette époque
        perm = pt.randperm(N_train)
        
        progress_bar = tqdm(range(0, N_train, batch_size), 
                           desc=f"Bucket {bucket_idx+1}, Epoch {epoch+1}/{num_epochs}", 
                           leave=False)
        
        for batchInd in progress_bar:
            batch_end = min(batchInd + batch_size, N_train)
            batch_indices = perm[batchInd:batch_end]
            
            BatchEventsHot = EvTens[batch_indices]
            BatchEventsInd = EvIndTens[batch_indices]
            BatchTimes = timeTensor[batch_indices]
            BatchTMax = tMaxTensor[batch_indices]
            BatchMask = mask[batch_indices]

            optimizer.zero_grad()
            
            # Forward pass
            lambOuts, CLows, Cbars, deltas, OutGates = net.forward(
                BatchEventsHot, BatchMask, BatchTimes
            )
            
            # Calculate losses
            LMC, _, _ ,individual_mc_losses= net.MC_Loss(
                BatchTimes, BatchTMax, CLows, Cbars, deltas, OutGates,
                Nsamples=N_seq_Max
            )
            LogLikeLoss, individual_log_losses = net.logLoss(BatchEventsInd, BatchMask, lambOuts)

            
            loss = LogLikeLoss + LMC
            loss.backward()
            optimizer.step()

            individual_total_losses = [mc + log for mc, log in zip(individual_mc_losses, individual_log_losses)]
            # MODIFICATION: Mapper chaque loss à son indice original
            for i, total_loss in enumerate(individual_total_losses):
                original_idx = batch_indices[i].item()
                epoch_losses_dict[original_idx] = total_loss       

            
            epoch_loss += loss.item()*(BatchEventsInd.shape[0])
            progress_bar.set_postfix({'Loss': f"{loss.item():.4f}"})
        
        # Calculer la loss per event pour cette époque
        train_loss_per_event = epoch_loss / np.sum(SeqLengthData)
        bucket_train_losses.append(train_loss_per_event)
        epoch_total_losses = [epoch_losses_dict[i] for i in range(N_train)]
        # NOUVELLE PARTIE: Évaluer sur validation après chaque époque
        val_loss = evaluate_on_validation(
            net, EvTens_val, EvIndTens_val, timeTensor_val, tMaxTensor_val, mask_val
        )
        val_loss_per_event = val_loss / np.sum(SeqLengthData_val)
        bucket_val_losses.append(val_loss_per_event)
        sequence_losses_per_epoch[epoch] = epoch_total_losses
        
        print(f"    Epoch {epoch+1}: Train loss/event: {train_loss_per_event:.6f}, Val loss/event: {val_loss_per_event:.6f}")
    
    return bucket_train_losses, bucket_val_losses, sequence_losses_per_epoch

# ============================================================================
# STRATÉGIE 1: CURRICULUM LEARNING (ordre selon le loss per event de l'epoch 2)
# ============================================================================
sequence_data = np.load('sequence_losses_per_epochV5.npy', allow_pickle=True).item()
with h5py.File("RetweetTrainData.h5", "r") as fl:
    SeqLengthData = np.array(fl["SeqLengthData"])
sequence_lengths = SeqLengthData[:4000]
epoch_2_losses = sequence_data[1]/sequence_lengths

def create_curriculum_bucketsV2(EventsData, SeqLengthData, timesData, timeMaxData, epoch_2_losses, num_buckets=4):
    """
    Curriculum Learning: Divise selon les losses de l'epoch 2 (ordonné) avec baby steps
    """
    # Trier les indices par losses croissantes
    sorted_indices = np.argsort(epoch_2_losses)
    
    # Diviser en buckets de base
    N_sequences = len(epoch_2_losses)
    bucket_size = N_sequences // num_buckets
    
    base_buckets = []
    for i in range(num_buckets):
        start_idx = i * bucket_size
        if i == num_buckets - 1:
            end_idx = N_sequences
        else:
            end_idx = (i + 1) * bucket_size
        base_buckets.append(sorted_indices[start_idx:end_idx])
    
    # Créer les buckets cumulatifs (baby steps)
    curriculum_buckets = []
    cumulative_indices = np.array([], dtype=int)
    
    for i, bucket in enumerate(base_buckets):
        cumulative_indices = np.concatenate([cumulative_indices, bucket])
        bucket_data = {
            'EventsData': EventsData[cumulative_indices],
            'SeqLengthData': SeqLengthData[cumulative_indices],
            'timesData': timesData[cumulative_indices],
            'timeMaxData': timeMaxData[cumulative_indices],
            'indices': cumulative_indices.copy()
        }
        curriculum_buckets.append(bucket_data)
    
    return curriculum_buckets

# ============================================================================
# STRATÉGIE 2: RANDOM BUCKETS (mêmes tailles, ordre aléatoire)
# ============================================================================

def create_random_buckets(EventsData, SeqLengthData, timesData, timeMaxData, num_buckets=4, seed=42):
    """
    Sélection laléatoire cumulative  
    """
    np.random.seed(seed)
    N_sequences = len(SeqLengthData)
    
    # Mélanger aléatoirement les indices
    shuffled_indices = np.random.permutation(N_sequences)
    
    # Diviser en buckets de même taille
    bucket_size = N_sequences // num_buckets
    
    base_buckets = []
    for i in range(num_buckets):
        start_idx = i * bucket_size
        if i == num_buckets - 1:
            end_idx = N_sequences
        else:
            end_idx = (i + 1) * bucket_size
        base_buckets.append(shuffled_indices[start_idx:end_idx])
    
    # Créer les buckets cumulatifs (baby steps)
    random_buckets = []
    cumulative_indices = np.array([], dtype=int)
    
    for i, bucket in enumerate(base_buckets):
        cumulative_indices = np.concatenate([cumulative_indices, bucket])
        bucket_data = {
            'EventsData': EventsData[cumulative_indices],
            'SeqLengthData': SeqLengthData[cumulative_indices],
            'timesData': timesData[cumulative_indices],
            'timeMaxData': timeMaxData[cumulative_indices],
            'indices': cumulative_indices.copy()
        }
        random_buckets.append(bucket_data)
    
    return random_buckets

# ============================================================================
# STRATÉGIE 3: PROGRESSIVE RANDOM (25%, 50%, 75%, 100%) totally random
# ============================================================================

def create_progressive_buckets(EventsData, SeqLengthData, timesData, timeMaxData, num_buckets=4, seed=42):
    """
    Progressive: Sélection aléatoire progressive (25%, 50%, 75%, 100%)
    """
    np.random.seed(seed)
    N_sequences = len(SeqLengthData)
    
    progressive_buckets = []
    
    for i in range(num_buckets):
        # Calculer le pourcentage: 25%, 50%, 75%, 100%
        percentage = (i + 1) / num_buckets
        num_sequences_in_bucket = int(N_sequences * percentage)
        
        # Générer de nouveaux indices aléatoires pour chaque bucket
        # Utiliser un seed différent pour chaque bucket
        np.random.seed(seed + i * 100)  # Seed différent pour chaque bucket
        bucket_indices = np.random.choice(N_sequences, num_sequences_in_bucket, replace=False)
        
        bucket_data = {
            'EventsData': EventsData[bucket_indices],
            'SeqLengthData': SeqLengthData[bucket_indices],
            'timesData': timesData[bucket_indices],
            'timeMaxData': timeMaxData[bucket_indices],
            'indices': bucket_indices.copy()
        }
        progressive_buckets.append(bucket_data)
    
    return progressive_buckets

# ============================================================================
# STRATÉGIE 1: CURRICULUM LEARNING (ordre selon la longueur, ordonné)
# ============================================================================
def create_curriculum_buckets(EventsData, SeqLengthData, timesData, timeMaxData, num_buckets=4):
    """
    Curriculum Learning: Divise selon la longueur (ordonné) avec baby steps
    """
    # Trier les indices par longueur croissante
    sorted_indices = np.argsort(SeqLengthData)
    
    # Diviser en buckets de base
    N_sequences = len(SeqLengthData)
    bucket_size = N_sequences // num_buckets
    
    base_buckets = []
    for i in range(num_buckets):
        start_idx = i * bucket_size
        if i == num_buckets - 1:
            end_idx = N_sequences
        else:
            end_idx = (i + 1) * bucket_size
        base_buckets.append(sorted_indices[start_idx:end_idx])
    
    # Créer les buckets cumulatifs (baby steps)
    curriculum_buckets = []
    cumulative_indices = np.array([], dtype=int)
    
    for i, bucket in enumerate(base_buckets):
        cumulative_indices = np.concatenate([cumulative_indices, bucket])
        bucket_data = {
            'EventsData': EventsData[cumulative_indices],
            'SeqLengthData': SeqLengthData[cumulative_indices],
            'timesData': timesData[cumulative_indices],
            'timeMaxData': timeMaxData[cumulative_indices],
            'indices': cumulative_indices.copy()
        }
        curriculum_buckets.append(bucket_data)
    
    return curriculum_buckets

# ============================================================================
# FONCTION PRINCIPALE D'ENTRAÎNEMENT ET COMPARAISON
# ============================================================================

def compare_training_strategies(file_path="RetweetTrainData.h5", dev_file_path="RetweetDevData.h5", 
                              N_train=4000, num_buckets=4, hD=16, lr=0.001, epochs_per_bucket=5):
    """
    Compare les 4 stratégies d'entraînement
    MODIFICATION: Validation loss calculée après chaque époque
    """
    print("Chargement des données...")
    
    # Charger les données d'entraînement
    with h5py.File(file_path, "r") as fl:
        EventsData = np.array(fl["EventsData"])[:N_train]
        timesData = np.array(fl["TimesData"])[:N_train]
        timeMaxData = np.array(fl["TimeMaxData"])[:N_train]
        SeqLengthData = np.array(fl["SeqLengthData"])[:N_train]
    
    # Charger les données de validation
    with h5py.File(dev_file_path, "r") as fl:
        EventsData_dev = np.array(fl["EventsData"])
        timesData_dev = np.array(fl["TimesData"])
        timeMaxData_dev = np.array(fl["TimeMaxData"])
        SeqLengthData_dev = np.array(fl["SeqLengthData"])
    
    print(f"Données chargées: {N_train} train, {len(EventsData_dev)} validation\n")
    
    # Créer les trois types de buckets
    print("Création des buckets pour les 4 stratégies...")
    curriculum_bucketsV2 = create_curriculum_bucketsV2(EventsData, SeqLengthData, timesData, timeMaxData, epoch_2_losses, num_buckets)
    random_buckets = create_random_buckets(EventsData, SeqLengthData, timesData, timeMaxData, num_buckets, seed=42)
    progressive_buckets = create_progressive_buckets(EventsData, SeqLengthData, timesData, timeMaxData, num_buckets, seed=42)
    curriculum_buckets= create_curriculum_buckets(EventsData, SeqLengthData, timesData, timeMaxData, num_buckets)
    
    strategies = [
        ("Curriculum LearningV2", curriculum_bucketsV2),
        ("Random Buckets", random_buckets), 
        ("Progressive Random", progressive_buckets),
        ("Curriculum Learning", curriculum_buckets)
    ]
    
    # Résultats pour chaque stratégie
    all_results = {}
    
    # Créer le dossier pour sauvegarder les modèles
    os.makedirs('Comparison_ModelsV6', exist_ok=True)
    
    for strategy_name, buckets in strategies:
        print(f"\n{'='*80}")
        print(f"ENTRAÎNEMENT: {strategy_name}")
        print('='*80)
        
        # Analyser les buckets
        for i, bucket in enumerate(buckets):
            lengths = bucket['SeqLengthData']
            print(f"Bucket {i+1}: {len(bucket['indices'])} séquences, "
                  f"longueurs [{lengths.min()}-{lengths.max()}], moyenne: {lengths.mean():.1f}")
        
        # Initialiser le modèle
        net = CTLSTM(K=3, hD=hD).double()
        optimizer = pt.optim.Adam(net.parameters(), lr=lr)
        
        # Variables pour tracking
        train_losses_per_event = []
        val_losses_per_event = []
        

        # Entraîner bucket par bucket
        for bucket_idx, bucket_data in enumerate(buckets):
            print(f"\nBucket {bucket_idx+1}/{num_buckets}:")
            
            # Préparer les tenseurs pour ce bucket
            EvTens, EvIndTens, timeTensor, tMaxTensor, mask, SeqLengthData_bucket = prepare_bucket_tensors(bucket_data)
            
            # Nombre de séquences de validation = nombre de séquences train / 4
            n_val_sequences = len(bucket_data['indices']) // 4
            n_val_sequences = min(n_val_sequences, len(EventsData_dev))  # Ne pas dépasser la taille des données de validation
            
            # Préparer les données de validation (sous-échantillon)
            val_bucket_data = {
                'EventsData': EventsData_dev[:n_val_sequences],
                'SeqLengthData': SeqLengthData_dev[:n_val_sequences],
                'timesData': timesData_dev[:n_val_sequences], 
                'timeMaxData': timeMaxData_dev[:n_val_sequences],
            }
            EvTens_val, EvIndTens_val, timeTensor_val, tMaxTensor_val, mask_val, SeqLengthData_val = prepare_bucket_tensors(val_bucket_data)
            
            print(f"  Train: {len(bucket_data['indices'])} séquences")
            print(f"  Validation: {n_val_sequences} séquences")
            
            # Entraîner sur ce bucket (MODIFICATION: retourne aussi val losses)
            bucket_train_losses, bucket_val_losses, _= train_bucket(
                net, optimizer, EvTens, EvIndTens, timeTensor, tMaxTensor, mask, SeqLengthData_bucket,
                EvTens_val, EvIndTens_val, timeTensor_val, tMaxTensor_val, mask_val, SeqLengthData_val,
                bucket_idx, num_epochs=epochs_per_bucket, batch_size=10
            )
            
            # Ajouter les losses de ce bucket aux listes globales
            train_losses_per_event.extend(bucket_train_losses)
            val_losses_per_event.extend(bucket_val_losses)
            
        
        # Sauvegarder le modèle final
        model_path = f'Comparison_ModelsV6/{strategy_name.replace(" ", "_")}_finalV6.pt'
        pt.save(net, model_path)
        print(f"Modèle sauvegardé: {model_path}")
        
        # Stocker les résultats
        all_results[strategy_name] = {
            'train_losses_per_event': train_losses_per_event,
            'val_losses_per_event': val_losses_per_event
        }
    
    # Créer les graphiques de comparaison
    # create_comparison_plots(all_results, num_buckets, epochs_per_bucket)
    
    # Sauvegarder tous les résultats
    np.save('training_strategies_comparisonV6.npy', all_results)
    print("\nRésultats sauvegardés dans 'training_strategies_comparison.npy'")

    
    return all_results



# ============================================================================
# FONCTION PRINCIPALE
# ============================================================================

if __name__ == "__main__":
    print("Comparaison de 4 stratégies d'entraînement")
    print("=" * 50)
    
    # Lancer la comparaison
    results = compare_training_strategies(
        file_path="RetweetTrainData.h5",
        dev_file_path="RetweetDevData.h5",
        N_train=4000,
        num_buckets=4,
        hD=16,
        lr=0.001,
        epochs_per_bucket=5
    )
    
    print("Comparaison terminée!")

Comparaison de 4 stratégies d'entraînement
Chargement des données...
Données chargées: 4000 train, 2000 validation

Création des buckets pour les 4 stratégies...

ENTRAÎNEMENT: Curriculum LearningV2
Bucket 1: 1000 séquences, longueurs [50-264], moyenne: 101.0
Bucket 2: 2000 séquences, longueurs [50-264], moyenne: 113.5
Bucket 3: 3000 séquences, longueurs [50-264], moyenne: 115.1
Bucket 4: 4000 séquences, longueurs [50-264], moyenne: 108.9

Bucket 1/4:
  Train: 1000 séquences
  Validation: 250 séquences


    Epoch 1: Train loss/event: 851.159020, Val loss/event: 1458.094311


    Epoch 2: Train loss/event: 199.119487, Val loss/event: 428.490496


    Epoch 3: Train loss/event: 76.295929, Val loss/event: 204.214286


    Epoch 4: Train loss/event: 41.942284, Val loss/event: 120.278547


...

    Epoch 1: Train loss/event: 7.094023, Val loss/event: 6.946991


    Epoch 2: Train loss/event: 6.867474, Val loss/event: 6.794487


    Epoch 3: Train loss/event: 6.760833, Val loss/event: 6.704490


    Epoch 4: Train loss/event: 6.681130, Val loss/event: 6.649036


    Epoch 5: Train loss/event: 6.626893, Val loss/event: 6.570908
Modèle sauvegardé: Comparison_ModelsV6/Curriculum_Learning_finalV6.pt

Résultats sauvegardés dans 'training_strategies_comparison.npy'
Comparaison terminée!
